In [56]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [57]:
mp_holistic = mp.solutions.holistic #that is holistic model
mp_drawing = mp.solutions.drawing_utils #that is for drawing utilites

In [58]:
def mediapipe_detection(frame , model):
    frame  = cv2.cvtColor(frame , cv2.COLOR_BGR2RGB) #so the model is expection RGB ??
    frame.flags.writeable = False #so  that make our live much easier
    results = model.process(frame)
    frame.flags.writeable = True
    frame = cv2.cvtColor(frame , cv2.COLOR_RGB2BGR) #as the normal format is BGR
    return frame , results


In [59]:
def draw_landmarks(frame, results):
    mp_drawing.draw_landmarks(frame, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) 
    mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) 
    mp_drawing.draw_landmarks(frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)


In [60]:
def draw_styled_landmarsks(frame, results):
    mp_drawing.draw_landmarks(frame, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),#joint 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)#lines
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                            mp_drawing.DrawingSpec(color=(80,110,10), thickness=2, circle_radius=4), 
                            mp_drawing.DrawingSpec(color=(80,256,121), thickness=2, circle_radius=4)
                             )  
    # Draw left hand connections
    mp_drawing.draw_landmarks(frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                            mp_drawing.DrawingSpec(color=(80,110,10), thickness=2, circle_radius=4), 
                            mp_drawing.DrawingSpec(color=(80,256,121), thickness=2, circle_radius=4)
                             )   
    # Draw right hand connections  
    mp_drawing.draw_landmarks(frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                            mp_drawing.DrawingSpec(color=(80,110,10), thickness=2, circle_radius=4), 
                            mp_drawing.DrawingSpec(color=(80,256,121), thickness=2, circle_radius=4)
                             )  

so to explain what is going on
-   first we get the frame we want 
-   then we pass that frame to fucntion that take the prediction and draw on frame
-   then we get the image with keypoint drawin  

In [61]:
cap = cv2.VideoCapture(0)
#so here we are setting mpmodel
with mp_holistic.Holistic(min_detection_confidence=0.5 , min_tracking_confidence=0.5) as holistic :
    while cap.isOpened():
        ret , frame = cap.read()
        
        #making detection
        frame , results = mediapipe_detection(frame , model=holistic)
        draw_styled_landmarsks(frame , results)

        cv2.imshow('openCV read' , frame)

        if cv2.waitKey(10) & 0xff == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

3. Extract Keypoint Values

In [62]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [63]:
results_test = extract_keypoints(results=results)

In [64]:
results_test.shape

(1662,)

okay that number of keypoint is what we need for each frame